In [158]:
#clean text and create stoplist
import jieba
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel
from gensim import corpora, models, similarities
import pandas as pd
import numpy as np
import csv
import re
import datetime
import time
print("start.",datetime.datetime.now())

def clean_text(text):
    text = text.replace("\n", " ") #newline
    text = re.sub(r"-", " ", text) #"-"
    text = re.sub(r"\d+/\d+/\d+", "", text) #timestamp
    text = re.sub(r"[\w]+@[\.\w]+", "", text) #ML
    text = re.sub(r"<.*?>", "", text) #pic
    text = re.sub(r"/[a-zA-Z]*[:\//\]*[A-Za-z0-9\-_]+\.+[A-Za-z0-9\.\/%&=\?\-_]+/i", "", text)
    pure_text = ""
    for letter in text:
        if letter.isalpha() or letter == " ":
            pure_text+=letter
    text = " ".join(word for word in pure_text.split() if len(word) > 1)
    return text

stoplist = pd.read_csv("/Users/CM/desktop/LDA2/stopzh.csv")["BodyText"].values
print("done.",datetime.datetime.now())

start. 2019-10-09 18:45:42.458587
done. 2019-10-09 18:45:42.467627


In [159]:
#csv→clean_text→clean_stoppoint→into_corpora
# Gensim
print("start.",datetime.datetime.now())

df = pd.read_csv("/Users/CM/Desktop/LDA2/ldazhtxt.csv")
df = df[["BodyText"]].dropna()

docs = df["BodyText"]
docs = docs.apply(lambda s: clean_text(s))

doclist = docs.values
texts1 = []

for i in doclist:
#    print(i)
    doclist2 = jieba.cut(i, cut_all=False,HMM=True)
    temp = " "
    for word in doclist2:
        if word not in stoplist:
            if word != '\t':
                if word != " ":
                    temp += word
                    temp += " "
#    print(temp)
#    print(temp.strip().split(" "))
    texts1.append(temp.strip().split(" "))

#texts1 = [[word for word in doc.split() if word not in stoplist] for doc in doclist1]
#texts1 = [[word for word in doc if word not in stoplist] for doc in doclist1]

#for i in range(20):
#    print(texts1[i])

texts = [doc for doc in texts1 if doc != []]

dictionary = corpora.Dictionary(texts)
corpus = [dictionary.doc2bow(text) for text in texts]

print("done.",datetime.datetime.now())

start. 2019-10-09 18:45:43.883433
done. 2019-10-09 18:46:43.041481


In [161]:
#training
print("start.",datetime.datetime.now())

num_topics = 10
chunksize = 6000
passes = 400
iterations = 400
eval_every = None

lda = gensim.models.ldamodel.LdaModel(corpus = corpus, id2word = dictionary,chunksize=chunksize, \
                                      alpha='auto', eta='auto',eval_every=eval_every, \
                                      iterations=iterations,passes = passes,num_topics = num_topics)

print("done.",datetime.datetime.now())
lda.print_topics(num_topics = num_topics, num_words = 20)

start. 2019-10-09 19:00:35.693924
done. 2019-10-09 19:06:14.010189


[(0,
  '0.011*"文库" + 0.008*"lncRNA" + 0.008*"激活" + 0.008*"转录" + 0.007*"表达" + 0.007*"sgRNA" + 0.007*"CRISPRCas" + 0.006*"凋亡" + 0.006*"应用" + 0.006*"MS" + 0.005*"筛选" + 0.005*"公司" + 0.004*"载体" + 0.004*"每个" + 0.004*"机制" + 0.004*"起始" + 0.004*"生物" + 0.004*"A" + 0.004*"CART" + 0.004*"B"'),
 (1,
  '0.006*"公司" + 0.005*"检测" + 0.004*"测试" + 0.004*"治疗" + 0.003*"疾病" + 0.003*"患者" + 0.003*"启动子" + 0.003*"医生" + 0.003*"诊断" + 0.003*"天花" + 0.003*"现在" + 0.002*"詹纳" + 0.002*"美国" + 0.002*"病毒" + 0.002*"疗法" + 0.002*"癌症" + 0.002*"预测" + 0.002*"中国" + 0.002*"问题" + 0.002*"提供"'),
 (2,
  '0.020*"规模" + 0.015*"工具" + 0.012*"全球" + 0.010*"万元" + 0.009*"中国" + 0.008*"市场" + 0.006*"类型" + 0.006*"表" + 0.006*"发展" + 0.006*"不同" + 0.006*"分析" + 0.005*"介绍" + 0.005*"企业" + 0.005*"公司" + 0.005*"市场份额" + 0.005*"预测" + 0.005*"增长率" + 0.004*"未来" + 0.004*"毛利率" + 0.004*"应用领域"'),
 (3,
  '0.011*"CCR" + 0.011*"脱靶" + 0.009*"HIV" + 0.009*"突变" + 0.008*"治疗" + 0.008*"病毒" + 0.007*"小鼠" + 0.006*"基因组" + 0.006*"干细胞" + 0.005*"团队" + 0.005*"免疫" + 0.005*"艾滋病" + 0.00

In [79]:
print("Perplexity: ", lda.log_perplexity(corpus))

Perplexity:  -6.780045484780302


In [128]:
from jieba import analyse
#spa = []
#for i in doclist:
#    spa.append(i)
    
#analyse.extract_tags(spa, topK = 20, withWeight = False, allowPOS=())
with open("/Users/CM/Desktop/LDA2/ldazhtxt.csv") as f:
    lines = f.read()
    words = analyse.extract_tags(lines, topK = 30, allowPOS=('n'))
    print(' '.join(words))


基因 编辑 技术 细胞 基因组 人类 序列 蛋白 小鼠 系统 位点 病毒 科学家 婴儿 胚胎 碱基 生物 疾病 团队 问题 工具 转基因 艾滋病 靶向 方法 细菌 干细胞 基因治疗 利用 外源
